# Assignment 4 for Course 1MS041
Make         sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline         and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `mammography.mat` that you can download from [http://odds.cs.stonybrook.edu/mammography-dataset/](http://odds.cs.stonybrook.edu/mammography-dataset/). Below you can find the code to load this file into `X` and `Y`, you just need to put the file in your `data` folder. During mammography the doctor would be looking for something called `calcification`, which is calcium deposits in the breast tissue and is used as an indicator of cancer. In this dataset the `X` corresponds to some measurements, there are 6 features. The `Y` is a 0-1 label where 1 represents calcification and 0 does not.

1. [3p] Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, I have prepared a shuffling with a fixed seed, this way I can make sure that we all did the same splits. That is [train,test,validation] is the splitting layout.
2. [4p] Train a machine learning model on the training data (you are free to choose it yourself). Hint: you could always try `LogisticRegression`, but for it to work well you would need `StandardScaler` and put everything in a `Pipeline`.
3. [3p] Use the classification report from `Utils` and compute the intervals for precision-recall etc on the test set with `union_bound correction` with 95% confidence.
4. [3p] You are interested in minimizing the average cost of your classifier, but first we must define it:
    * If someone is calcified but classified as not, we say it costs 30 **(this is the worst scenario)** 
    * If someone is not calcified but classified as calcified, we say it costs 5 **(this probably only costs extra investigation)**
    * If someone is calcified but classified as calcified, costs 0 **(We did the right thing, no cost)**
    * If someone is not calcified but classified as not, costs 0 **(We did the right thing, no cost)**.

complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). What would be the cost of having a model that always classifies someone as not calcified on the test set?

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, we do that by checking say 100 evenly spaced proposal thresholds between 0 and 1, and use our testing data to compute the cost.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 95% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted, if we define the random variable
$$
    C = 30(1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
$$
then $C$ denotes the cost of a randomly chosen patient. In the above we are estimating $\mathbb{E}[C]$ using the empirical mean. However, since the number of calcifications in the population is fairly small and our classifier probably has a fairly high precision for the $0$ class, then $C$ should have fairly small variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [1]:
import scipy.io as so
import numpy as np
data = so.loadmat('data/mammography.mat')

np.random.seed(0)
shuffle_index = np.arange(0,len(data['X']))
np.random.shuffle(shuffle_index)

X = data['X'][shuffle_index,:]
Y = data['y'][shuffle_index,:].flatten()

In [2]:

# Part 1

# Split the X,Y into three parts, make sure that the sizes are
# (6709, 6), (1677, 6), (2797, 6), (6709,), (1677,), (2797,)
import sklearn.model_selection as skl_ms

#X_train_test, X_valid, = skl_ms.train_test_split(X,train_size = 0.75 ,shuffle = False)
#Y_train_test, Y_valid, = skl_ms.train_test_split(Y,train_size = 0.75 ,shuffle = False)

#print(X_train_test.shape, X_valid.shape, Y_train_test.shape, Y_valid.shape)

#X_train, X_test = skl_ms.train_test_split(X_train_test,test_size = 0.2 ,shuffle = False)
#Y_train, Y_test = skl_ms.train_test_split(Y_train_test,test_size = 0.2 ,shuffle = False)

X_train, X_test, X_valid, Y_train, Y_test, Y_valid = X[:6709],X[6709:6709+1677],X[6709+1677:],Y[:6709],Y[6709:6709+1677],Y[6709+1677:]
print(X_train.shape, X_test.shape, X_valid.shape, Y_train.shape, Y_test.shape, Y_valid.shape)


(6709, 6) (1677, 6) (2797, 6) (6709,) (1677,) (2797,)


In [3]:

# Part 2
# Use X_train,Y_train to train a model from sklearn. Make sure it has the predict_proba function
# for instance LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
model = Pipeline([('scaler', StandardScaler()), ('LogReg', LogisticRegression())])
model.fit(X_train, Y_train)
print(model.fit(X_train, Y_train).score(X_test, Y_test))
Y_pred = model.predict(X_test)


0.9815146094215862


In [4]:

# Part 3

# Compute precision and recall on the test set using 
from Utils import classification_report_interval
print(classification_report_interval(Y_test,Y_pred,alpha=0.05))
# Each precision and recall should be a tuple, for instance you can write
# precision0 = (0.9,0.95)

precision0 = (0.94,1.00)
recall0 = (0.96,1.00)
precision1 = (0.33,1.00)
recall1 = (0.09,0.62)

#print(type(precision0),type(recall0),type(precision0),type(recall1))

# The code below will check that you supply the proper type
assert(type(precision0) == tuple)
assert(len(precision0) == 2)
assert(type(recall0) == tuple)
assert(len(recall0) == 2)
assert(type(precision1) == tuple)
assert(len(precision1) == 2)
assert(type(recall1) == tuple)
assert(len(recall1) == 2)

            labels           precision             recall

                 0  0.98 : [0.94,1.00] 1.00 : [0.96,1.00]
                 1  0.70 : [0.33,1.00] 0.36 : [0.09,0.62]

          accuracy                                        0.98 : [0.94,1.00]



In [5]:

# Part 4
"""4. [3p] You are interested in minimizing the average cost of your classifier, but first we must define it:
    * If someone is calcified but classified as not, we say it costs 30 **(this is the worst scenario)** 
    * If someone is not calcified but classified as calcified, we say it costs 5 **(this probably only costs extra investigation)**
    * If someone is calcified but classified as calcified, costs 0 **(We did the right thing, no cost)**
    * If someone is not calcified but classified as not, costs 0 **(We did the right thing, no cost)**.
complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold
(recall our precision recall lecture and the `predict_proba` function from trained models).
"""
def cost(model,threshold,X,Y):
    pred_proba = model.predict_proba(X)[:,1]
    # true label = Y, predicted label = predictions
    predictions = (pred_proba >= threshold)*1
    cost_value = 0
    for i in range(len(Y)):
        if Y[i]==predictions[i]:
            cost_value = cost_value + 0
        elif Y[i] ==1 and predictions[i]==0:
            cost_value = cost_value + 30
        elif Y[i]==0 and predictions[i]==1:
            cost_value = cost_value +5
    cost_per_person = 1/len(Y)*cost_value
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost (cost per person)
    
    return cost_per_person

def cost_array(model,threshold,X,Y):
    pred_proba = model.predict_proba(X)[:,1]
    # true label = Y, predicted label = predictions
    predictions = (pred_proba >= threshold)*1
    cost_value = []
    for i in range(len(Y)):
        if Y[i]==predictions[i]:
            cost_value.append(0)
        elif Y[i] ==1 and predictions[i]==0:
            cost_value.append(30)
        elif Y[i]==0 and predictions[i]==1:
            cost_value.append(5)
    #cost_per_person = 1/len(Y)*cost_value
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost (cost per person)
    
    return cost_value

In [6]:

# Part 4
"""What would be the cost of having a model that always classifies someone as not calcified on the test set?"""
# Fill in the naive cost of the model that would classify all as non-calcified on the test set

# true label = Y, predicted label = predictions
naive_predictions = [0]*len(Y)
cost_value = 0
for i in range(len(Y)):
    if Y[i]!=naive_predictions[i]:
        cost_value = cost_value + 30
naive_cost = 1/len(Y)*cost_value
print(naive_cost)

0.6974872574443352


In [11]:

# Part 5
"""Now, we wish to select the threshold of our classifier that minimizes the cost,
we do that by checking say 100 evenly spaced proposal thresholds between 0 and 1,
and use our testing data to compute the cost.
"""
import numpy as np
threshold = np.linspace(0,1,100)
print(threshold)
save_cost = []
for i in range(len(threshold)):
    save_cost.append(cost(model,threshold[i],X_test,Y_test))

optimal_threshold = threshold[np.argmin(save_cost)]
cost_at_optimal_threshold = np.min(save_cost)

print(optimal_threshold)
print(cost_at_optimal_threshold,optimal_threshold)

[0.         0.01010101 0.02020202 0.03030303 0.04040404 0.05050505
 0.06060606 0.07070707 0.08080808 0.09090909 0.1010101  0.11111111
 0.12121212 0.13131313 0.14141414 0.15151515 0.16161616 0.17171717
 0.18181818 0.19191919 0.2020202  0.21212121 0.22222222 0.23232323
 0.24242424 0.25252525 0.26262626 0.27272727 0.28282828 0.29292929
 0.3030303  0.31313131 0.32323232 0.33333333 0.34343434 0.35353535
 0.36363636 0.37373737 0.38383838 0.39393939 0.4040404  0.41414141
 0.42424242 0.43434343 0.44444444 0.45454545 0.46464646 0.47474747
 0.48484848 0.49494949 0.50505051 0.51515152 0.52525253 0.53535354
 0.54545455 0.55555556 0.56565657 0.57575758 0.58585859 0.5959596
 0.60606061 0.61616162 0.62626263 0.63636364 0.64646465 0.65656566
 0.66666667 0.67676768 0.68686869 0.6969697  0.70707071 0.71717172
 0.72727273 0.73737374 0.74747475 0.75757576 0.76767677 0.77777778
 0.78787879 0.7979798  0.80808081 0.81818182 0.82828283 0.83838384
 0.84848485 0.85858586 0.86868687 0.87878788 0.88888889 0.89898

In [8]:

# Part 6
"""With your newly computed threshold value, compute the cost of putting this model in production
by computing the cost using the validation data. Also provide a confidence interval of the cost
using Hoeffdings inequality with a 95% confidence.
"""
cost_at_optimal_threshold_validation = cost(model,optimal_threshold,X_valid,Y_valid)
print(cost_at_optimal_threshold_validation)
#cost_arr = []
#cost_arr.append(cost_at_optimal_threshold_validation)
cost_arr = cost_array(model,optimal_threshold,X_valid,Y_valid)
print(cost_arr)
# Report the cost interval as a tuple cost_interval = (a,b)
from Utils import compute_confidence_interval_bounded
#cost_array = cost(model, optimal_threshold, X_valid, Y_valid, return_array=True)
cost_interval = compute_confidence_interval_bounded(cost_arr,0.05,0,30)
print(cost_interval)

#(0.68,0.73)

# The code below will tell you if you filled in the intervals correctly
assert(type(cost_interval) == tuple)
assert(len(cost_interval) == 2)


0.2913836253128352
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 30, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [9]:

# Part 7
"""Let t be the threshold you found and f the model you fitted, if we define the random variable
$$
    C = 30(1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
$$
then C denotes the cost of a randomly chosen patient.
In the above we are estimating $\mathbb{E}[C]$ using the empirical mean.
However, since the number of calcifications in the population is fairly small and
our classifier probably has a fairly high precision for the 0 class, then C should have fairly small variance.
Compute the empirical variance of C on the validation set.
"""
variance = 0
pred_proba = model.predict_proba(X_valid)[:,1]
Y_pred = (pred_proba >= optimal_threshold)*1
costs = 30*(1-Y_pred)*Y_valid+5*(1-Y_valid)*Y_pred

for i in range(len(Y_valid)):
    variance = variance + (costs[i]-cost_at_optimal_threshold_validation)**2
variance_of_C = 1/len(Y_valid)*variance

print(variance_of_C)




6.198613637958718


In [10]:

# Part 7
"""What would be the confidence interval if we used Bennett's inequality instead of Hoeffding
in point 6 but with the computed empirical variance as our guess for the variance?"""
# A 95% confidence interval of the random variable C using Bennett's inequality
from Utils import bennett_epsilon,print_confidence_interval
eps2 = bennett_epsilon(len(Y_valid),30,variance_of_C,0.025)
print_confidence_interval(cost_at_optimal_threshold_validation,eps2)

interval_of_C = (0.15,0.43)

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

Numerical error 1.5959455978986625e-16
[-0.07,0.65]
